In [1]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

2022-03-23 10:15:10.788945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-23 10:15:10.789061: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from RainPredictionMachine.data import CleanDataRpm
cleaner = CleanDataRpm()
df = cleaner.clean_data(2)

In [3]:
df.head()

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid,Dir_vento,Rajada_vento,Vel_vento,Estaçao,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,923.1,923.1,922.1,0.0,22.0,20.5,22.2,21.8,20.5,...,92.0,172.0,2.8,0.4,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.0,2020-01-01 00:00:00+00:00,nao chove
1,0.0,923.7,923.7,923.1,0.0,22.1,20.7,22.3,22.0,20.8,...,92.0,101.0,1.8,0.9,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.0,2020-01-01 01:00:00+00:00,nao chove
2,0.0,923.6,923.8,923.6,0.0,23.2,20.1,23.5,22.1,20.8,...,82.0,7.0,3.3,1.2,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.0,2020-01-01 02:00:00+00:00,nao chove
3,0.0,923.1,923.6,923.1,0.0,23.3,19.5,23.6,23.1,20.1,...,79.0,6.0,3.0,1.2,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.0,2020-01-01 03:00:00+00:00,nao chove
4,0.0,922.6,923.1,922.6,0.0,22.9,19.0,23.6,22.8,19.5,...,79.0,345.0,3.5,0.6,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.0,2020-01-01 04:00:00+00:00,nao chove


In [4]:
df_test = df.set_index(['Estaçao', 'datahora'])

In [22]:
features = ['Pres',
            'Pres_max',
            'Pres_min',
            'Radiacao',
            'Temp',
            'Temp_orvalho',
            'Temp_max',
            'Temp_min',
            'Temp_orvalho_max',
            'Temp_orvalho_min',
            'Umid_max',
            'Umid_min',
            'Umid',
            'Rajada_vento',
            'Vel_vento',
            'Dir_vento']
X = []
y = []

for est in df["Estaçao"].unique():
    X_temp = df[df["Estaçao"]==est][features].values
    y_temp = df[df["Estaçao"]==est]["classe_chuva"].values
    X.append(X_temp)
    y.append(y_temp)
    
X = np.array(X)
y = np.array(y)

In [23]:
X.shape

(2, 8784, 16)

In [26]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label = LabelEncoder()
y_enc = label.fit_transform(y.reshape((-1,)))
y_cat = to_categorical(y_enc)
y_cat = y_cat.reshape(2,8784,-1)
y_cat.shape

(2, 8784, 4)

In [28]:
train_size = int(X.shape[1]*0.8)
X_train = X[:,:train_size,:]
X_test = X[:,train_size:,:]
y_train = y_cat[:,:train_size,:]
y_test = y_cat[:,train_size:,:]

In [33]:
X_train.shape

(2, 7027, 16)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer



#----------------scaling pipeline----------------
scaling_pipe = Pipeline([
    ('stdscaler', StandardScaler()),
])

In [ ]:
#----------------column transformer----------------
#realizando as operações em paralelo
col_trans = ColumnTransformer([
    ('scaling ', scaling_pipe,[ 'Pres',
                                'Pres_max',
                                'Pres_min',
                                'Radiacao',
                                'Temp',
                                'Temp_orvalho',
                                'Temp_max',
                                'Temp_min',
                                'Temp_orvalho_max',
                                'Temp_orvalho_min',
                                'Umid_max',
                                'Umid_min',
                                'Umid',
                                'Rajada_vento',
                                'Vel_vento',
                                'Dir_vento']), 
    ('pass', 'passthrough', ['Latitude','Longitude','Altitude'])])

full_pipe = Pipeline([
    ('column_stransformer', col_trans),
#    ("deep_learning" , RNN_model ),
])
full_pipe

In [34]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype= 'float32', padding= 'post', value=-10000) 

In [35]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

norm = Normalization()
norm.adapt(X_train_pad)

2022-03-23 10:59:18.127366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-23 10:59:18.127523: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-23 10:59:18.127624: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-9BMAD68): /proc/driver/nvidia/version does not exist
2022-03-23 10:59:18.129328: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [47]:
from tensorflow.keras.layers import Masking

model = Sequential()

model.add(Masking(mask_value=-10000,input_shape=X_train_pad.shape[1:]))
model.add(Normalization())

model.add(LSTM(units=20, activation='tanh',return_sequences = True))
model.add(Dense(10, activation="tanh"))
model.add(Dense(4, activation="softmax"))

model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_5 (Masking)         (None, 7027, 16)          0         
                                                                 
 normalization_3 (Normalizat  (None, 7027, 16)         33        
 ion)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 7027, 20)          2960      
                                                                 
 dense_10 (Dense)            (None, 7027, 10)          210       
                                                                 
 dense_11 (Dense)            (None, 7027, 4)           44        
                                                                 
Total params: 3,247
Trainable params: 3,214
Non-trainable params: 33
___________________________________________________

In [43]:
y_train.shape

(2, 7027, 4)

In [44]:
X_train_pad.shape

(2, 7027, 16)

In [48]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train_pad, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/10
1/1 [==============================] - 23s 23s/step - loss: 1.4191 - accuracy: 0.0743 - val_loss: 1.2259 - val_accuracy: 0.0865
Epoch 2/10
1/1 [==============================] - 8s 8s/step - loss: 1.2083 - accuracy: 0.0990 - val_loss: 1.1462 - val_accuracy: 0.2247
Epoch 3/10
1/1 [==============================] - 7s 7s/step - loss: 1.1346 - accuracy: 0.2123 - val_loss: 1.0886 - val_accuracy: 0.6318
Epoch 4/10
1/1 [==============================] - 7s 7s/step - loss: 1.0783 - accuracy: 0.6533 - val_loss: 1.0550 - val_accuracy: 0.6441
Epoch 5/10
1/1 [==============================] - 7s 7s/step - loss: 1.0442 - accuracy: 0.6647 - val_loss: 1.0237 - val_accuracy: 0.6676
Epoch 6/10
1/1 [==============================] - 7s 7s/step - loss: 1.0147 - accuracy: 0.6831 - val_loss: 0.9968 - val_accuracy: 0.8548
Epoch 7/10
1/1 [==============================] - 7s 7s/step - loss: 0.9888 - accuracy: 0.8261 - val_loss: 0.9720 - val_accuracy: 0.8698
Epoch 8/10
1/1 [=======================